In [2]:
import tensorflow as tf 
import numpy as np 
from matplotlib import pyplot as plt 

2023-04-27 09:45:32.342517: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        loss = logs["loss"]
        if loss <= self.threshold:
            self.model.stop_training = True


In [4]:
epoch_loss = []

class LogEpochLoss(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        epoch_loss.append(logs["loss"])

In [5]:
from tensorflow.keras.datasets import mnist 

In [7]:
(trainX, trainy), (testX, testy) = mnist.load_data()

X = trainX
y = trainy

y = tf.keras.utils.to_categorical(y)


print(X.shape)

X = np.expand_dims(X, axis=-1)

X = tf.image.resize(X,  [32,32]) 

X = X[0:500]
y = y[0:500]
print("X size:", X.shape)
print("Y size:", y.shape)

input_shape = X[0].shape
input_shape = input_shape + (1,)

(60000, 28, 28)
X size: (500, 32, 32, 1)
Y size: (500, 10)


In [8]:
base_model = tf.keras.applications.EfficientNetB0(input_shape = (32, 32, 1), include_top = False, weights = None )

In [9]:
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = 1000, activation = 'relu'),
    tf.keras.layers.Dense(units = 10, activation = 'softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 1, 1, 1280)       4048991   
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1000)              1281000   
                                                                 
 dense_1 (Dense)             (None, 10)                10010     
                                                                 
Total params: 5,340,001
Trainable params: 5,297,982
Non-trainable params: 42,019
_________________________________________________________________


In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs = 10, batch_size = 10, verbose = 1)

Epoch 1/10
10/10 [==============================] - 19s 210ms/step - loss: 0.2321 - accuracy: 0.9300
Epoch 2/10
10/10 [==============================] - 2s 217ms/step - loss: 0.1807 - accuracy: 0.9500
Epoch 3/10
10/10 [==============================] - 2s 212ms/step - loss: 0.1073 - accuracy: 0.9800
Epoch 4/10
10/10 [==============================] - 2s 224ms/step - loss: 0.0892 - accuracy: 0.9740
Epoch 5/10
10/10 [==============================] - 2s 204ms/step - loss: 0.0709 - accuracy: 0.9780
Epoch 6/10
10/10 [==============================] - 2s 233ms/step - loss: 0.0862 - accuracy: 0.9720
Epoch 7/10
10/10 [==============================] - 2s 209ms/step - loss: 0.1050 - accuracy: 0.9620
Epoch 8/10
10/10 [==============================] - 2s 209ms/step - loss: 0.0746 - accuracy: 0.9760
Epoch 9/10
10/10 [==============================] - 3s 256ms/step - loss: 0.1552 - accuracy: 0.9640
Epoch 10/10
10/10 [==============================] - 2s 207ms/step - loss: 0.1883 - accuracy: 0.954

In [14]:
base_model.trainable = False 

In [15]:
base_model.save_weights('models/')

In [16]:
unit_range = np.arange(1000, 100000, 1000)

In [17]:
model = tf.keras.models.Sequential([ 
    base_model,
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(units = 1000, activation = 'relu'),
    tf.keras.layers.Dense(units = 10, activation = 'softmax')
])
model.summary()

print(np.shape(model.layers[3].get_weights()[0]))


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 1, 1, 1280)       4048991   
                                                                 
 flatten_1 (Flatten)         (None, 1280)              0         
                                                                 
 dense_2 (Dense)             (None, 1000)              1281000   
                                                                 
 dense_3 (Dense)             (None, 10)                10010     
                                                                 
Total params: 5,340,001
Trainable params: 1,291,010
Non-trainable params: 4,048,991
_________________________________________________________________
(1000, 10)


In [18]:
initial_weights_1 = []
initial_weights_2 = []
final_weights_1 = []
final_weights_2 = []

In [19]:


for units in unit_range:
    model = tf.keras.models.Sequential([ 
        base_model,
        tf.keras.layers.Flatten(), 
        tf.keras.layers.Dense(units = units, activation = 'relu'),
        tf.keras.layers.Dense(units = 10, activation = 'softmax')
    ])
    initial_weights_1.append(model.layers[2].get_weights()[0])
    initial_weights_2.append(model.layers[3].get_weights()[0])
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X, y, epochs = 5, batch_size = 10,  verbose = 1)

    final_weights_1.append(model.layers[2].get_weights()[0])
    final_weights_2.append(model.layers[3].get_weights()[0])




Epoch 1/5
50/50 [==============================] - 8s 30ms/step - loss: 0.2173 - accuracy: 0.9420
Epoch 2/5
50/50 [==============================] - 1s 29ms/step - loss: 0.0425 - accuracy: 0.9840
Epoch 3/5
50/50 [==============================] - 2s 32ms/step - loss: 0.1431 - accuracy: 0.9780
Epoch 4/5
50/50 [==============================] - 2s 32ms/step - loss: 0.1104 - accuracy: 0.9760
Epoch 5/5
50/50 [==============================] - 2s 31ms/step - loss: 0.0260 - accuracy: 0.9900
Epoch 1/5
50/50 [==============================] - 8s 52ms/step - loss: 0.1983 - accuracy: 0.9460
Epoch 2/5
50/50 [==============================] - 3s 51ms/step - loss: 0.3498 - accuracy: 0.9740
Epoch 3/5
50/50 [==============================] - 3s 51ms/step - loss: 0.1311 - accuracy: 0.9820
Epoch 4/5
50/50 [==============================] - 3s 55ms/step - loss: 0.0522 - accuracy: 0.9800
Epoch 5/5
50/50 [==============================] - 3s 53ms/step - loss: 0.0766 - accuracy: 0.9900
Epoch 1/5
50/50 [===